In [0]:
import numpy as np
import pandas as pd
import scipy as sp
import cupy as cp
import time
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date as dt
from datetime import datetime as dtm
from datetime import timedelta as td

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### CUPY GBM

In [0]:
def randn(size,g=None):
    if not g:
        g = cp.cuda.curand.createGenerator(cp.cuda.curand.CURAND_RNG_QUASI_SOBOL64)
    element_size = size[0]*size[1] if len(size)>1 else size[0]
    if element_size % 2 == 0:
        out = cp.empty(element_size, dtype=np.float64)
        cp.cuda.curand.generateNormalDouble(g, out.data.ptr, out.size,0,1)
        cp.random.shuffle(out)
        return out.reshape(size)
    else:
        out = cp.empty((element_size + 1,), dtype=np.float64)
        cp.cuda.curand.generateNormalDouble(g, out.data.ptr, out.size,0,1)
        return out[:element_size].reshape(size)

In [0]:
g = cp.cuda.curand.createGenerator(cp.cuda.curand.CURAND_RNG_QUASI_DEFAULT)
cp.cuda.curand.setGeneratorOrdering(g,cp.cuda.curand.CURAND_ORDERING_QUASI_DEFAULT)

In [0]:
def GBM(today,expiry,S0,ir,div,vol,g=None,num=1000000,bb=True,skip=0,seed=42):
    steps = (expiry-today).days
    tenor = steps/365
    
    idx = np.arange(steps+1)
    den = np.insert(idx[1:],0,1)
    
    def linear_interp(x,y,new_x):
        return sp.interpolate.interp1d(x,y,"linear",fill_value="extrapolate")(new_x)
    
    def check_term(series,start=0):
        new = series.copy()
        if isinstance(series.index,pd.DatetimeIndex):
            new.index = (series.index-today).days
        if new.index[0]>start:
            new = pd.Series({start:new.iloc[0]}).append(new)
        elif new.inedx[0]<start:
            new.index[0] = start
        return new        
    
    if isinstance(ir,(float,np.float64)):
        ir_curve = cp.asarray(np.ones(len(idx))*ir)
    elif isinstance(ir,pd.Series):
        ir_ = check_term(ir,0)
        ir_curve = cp.asarray(linear_interp(ir_.index,ir_.index*ir_,idx)/den)
    
    if not div:div_curve = 0
    if isinstance(div,(float,np.float64)):
        div_curve = cp.asarray(np.ones(len(idx))*div)
    elif isinstance(div,pd.Series):
        div_ = check_term(div,0)
        div_curve = cp.asarray(linear_interp(div_.index,div_.index*div_,idx)/den)
        
    if isinstance(vol,(float,np.float64)):
        var_curve = cp.asarray(np.ones(len(idx))*np.square(vol))
    elif isinstance(vol,pd.Series):
        vol_ = check_term(vol,1)
        var_curve = cp.asarray(linear_interp(vol_.index,vol_.index*np.square(vol_),idx)/den)
    
    dt = cp.asarray(idx/365)
    sigma2t = cp.asarray(cp.diff(var_curve*dt,prepend=0))
    drift = cp.asarray(cp.diff((ir_curve-div_curve)*dt,prepend=0) - 0.5*sigma2t)
    stoch = cp.sqrt(sigma2t)

    if not g:
        g = cp.cuda.curand.createGenerator(cp.cuda.curand.CURAND_RNG_QUASI_DEFAULT)
        cp.cuda.curand.setGeneratorOrdering(g,cp.cuda.curand.CURAND_ORDERING_QUASI_DEFAULT)

    element_size = num*(steps+1)
    tar_size = element_size if element_size%2==0 else element_size+1
    bm = cp.empty(tar_size, dtype=np.float32)
    cp.cuda.curand.generateNormal(g, bm.data.ptr, bm.size,0,1)
    cp.random.shuffle(bm)
    bm = bm[:element_size].reshape((num,steps+1))
    bm[:,0] = 0

    return S0*cp.exp((drift + bm*stoch).cumsum(axis=1))

### Test

In [0]:
todaydt = dtm(2019,11,20)
expirydt = dtm(2020,11,20)
ir = 0.0260156595595066
vol = 0.114975694793223
weekdays = cp.array([x for x in range(1,367) if (todaydt+td(days=x)).isoweekday()<6])
coupon = cp.array([0.05*i/12 for i in range(1,13)])
call_barrier = 1.0
call_obs = cp.array([30,61,92,121,152,182,213,243,274,305,335,366])
df = cp.exp(-ir*cp.array(call_obs)/365)

In [0]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()

In [0]:
%%time
path = GBM(todaydt,expirydt,1,ir,None,vol,g=g,num=1000000,bb=True,skip=0,seed=None)

In [0]:
def Snowball(path,coupon,call_barrier,call_obs,ki_barrier,ki_obs,min_gain,max_gain,df,notional):
    obs = path[:,call_obs]>call_barrier #观察日是否赎回
    called = obs.any(axis=1)            #每条路径是否赎回
    called_value = cp.eye(len(call_obs))[obs[called].argmax(axis=1)]*coupon*df #赎回日所获coupon贴现

    ki = (path[~called][:,ki_obs]<ki_barrier).any(axis=1)                      #未赎回但敲入
    ki_value = (path[~called][ki][:,-1].clip(a_min=min_gain,a_max=max_gain)-1)*df[-1] #敲入损失贴现

    nki_value = (~ki).sum()*coupon[-1]*df[-1] #未赎回未敲入全额coupon贴现

    return (called_value.sum()+ki_value.sum()+nki_value)/path.shape[0]*notional #相加平均

In [0]:
Snowball(path,coupon,call_barrier,call_obs,0.7,weekdays,0.01,1,df,1e6)

### Repeat

In [0]:
class EquityOption:
    def __init__(self,):
        pass

    def with_schedule(self,today,expiry):
        if not isinstance(today,(dt,dtm)):
            raise TypeError("[MC.schedule] argument 'today' must be a data/datetime object")
        if not isinstance(expiry,(dt,dtm)):
            raise TypeError("[MC.schedule] argument 'expiry' must be a data/datetime object")
        self.today = today
        self.expiry = expiry
        self.steps = (expiry-today).days
        self.tenor = self.steps/365        
        self.idx = np.arange(self.steps+1)
        self.den = np.insert(self.idx[1:],0,1)
        self.len = self.steps+1
        self.tau = self.idx/365
        return self

    @staticmethod
    def linear_interp(x,y,new_x):
        return sp.interpolate.interp1d(x,y,"linear",fill_value="extrapolate")(new_x)

    @staticmethod
    def check_term(self,series,start=0):
        new = series.copy()
        if isinstance(series.index,pd.DatetimeIndex):
            new.index = (series.index-self.today).days
        if new.index[0]>start:
            new = pd.Series({start:new.iloc[0]}).append(new)
        elif new.inedx[0]<start:
            new.index[0] = start
        return new 

    def with_ir(self,ir):
        if isinstance(ir,(float,np.float64)):
            self.ir_curve = np.ones(self.len)*ir
        elif isinstance(ir,pd.Series):
            ir_ = self.check_term(ir,0)
            self.spot_curve = self.linear_interp(ir_.index,ir_.index*ir_,self.idx)/self.den
            self.ir_curve = np.diff(self.spot_curve*self.tau,prepend=self.spot_curve[0])/(1/365)
        return self
    
    def with_div(self,div):
        if isinstance(div,(float,np.float64)):
            self.div_curve = np.ones(self.len)*div
        elif isinstance(div,pd.Series):
            div_ = self.check_term(div,0)
            self.div_curve = self.linear_interp(div_.index,div_.index*div_,self.idx)/self.den
        return self

    def with_vol(self,vol):
        if isinstance(vol,(float,np.float64)):
            self.var_curve = np.ones(self.len)*np.square(vol)
        elif isinstance(vol,pd.Series):
            vol_ = self.check_term(vol,1)
            self.var_curve = self.linear_interp(vol_.index,vol_.index*np.square(vol_),self.idx)/self.den
        return self
    
    def calc_term(self):
        if not hasattr(self,"div_curve"):
            self.div_curve = 0        
        self.sigma2t = np.diff(self.var_curve*self.tau)
        self.drift = np.diff((self.ir_curve-self.div_curve)*self.tau)-0.5*self.sigma2t
        self.stoch = np.sqrt(self.sigma2t)
        self.df_curve = np.exp(-self.ir_curve*self.tau)
        return self.steps,self.drift,self.stoch,self.df_curve
        

In [0]:
class Snowball:
    def __init__(self,S0,coupon,call_barrier,call_obs,ki_barrier,ki_obs,min_gain,max_gain,notional,dtp):
        self.dtp = dtp

        self.S0 = dtp(S0)
        self.coupon =  cp.asarray(coupon,dtype=dtp)
        self.call_barrier = cp.asarray(call_barrier,dtype=dtp)
        self.call_obs = cp.asarray(call_obs,dtype=cp.bool)
        self.ki_barrier = cp.asarray(ki_barrier,dtype=dtp)
        self.ki_obs = cp.asarray(ki_obs,dtype=cp.bool)
        self.min_val = dtp(min_val)
        self.max_val = dtp(max_val)
        
        self.notional = dtp(notional)
        self._compile()

    def with_proc(self,steps,drift,stoch,df):
        self.steps = steps
        self.drift = cp.asarray(drift,dtype=self.dtp)
        self.stoch = cp.asarray(stoch,dtype=self.dtp)
        self.df = cp.asarray(df,dtype=self.dtp)
        return self

    def with_rng(self,qmc=False,double_float=False,skip=10,seed=None):
        self.dbl = double_float
        self.qmc = qmc
        self.seed = seed
        if qmc:
            if double_float: 
                gtype = cp.cuda.curand.CURAND_RNG_QUASI_SOBOL64                
            else: 
                gtype = cp.cuda.curand.CURAND_RNG_QUASI_SOBOL32                
        else:gtype = cp.cuda.curand.CURAND_RNG_PSEUDO_XORWOW
            
        if double_float:
            self.dtp = cp.float64
            self.gfunc = cp.cuda.curand.generateNormalDouble
        else:
            self.dtp = cp.float32
            self.gfunc = cp.cuda.curand.generateNormal

        self.g = cp.cuda.curand.createGenerator(gtype)
        cp.cuda.curand.setGeneratorOffset(self.g,skip)
        if not qmc and seed:
            cp.cuda.curand.setPseudoRandomGeneratorSeed(self.g,seed)
        return self

    def run(self,N_PATHS,N_ROUND,N_BLOCKS,N_THREADS):
        if not hasattr(self,"drift"):
            raise("[Snowball.run] process is not specified.")
        if not hasattr(self,"g"):
            raise("[Snowball.run] random generator is not specified.")
        total = cp.float64(0.0)
        output = cp.empty(N_PATHS, dtype=dtp)
        if self.seed: seeds = np.arange(self.seed,self.seed+N_ROUND,dtype=int)
        else: seeds = np.random.randint(100,10000,N_ROUND)
        for i in range(N_ROUND):
            cp.cuda.curand.setPseudoRandomGeneratorSeed(self.g,seeds[i])
            RS = cp.zeros(N_PATHS*self.steps, dtype=dtp)
            self.gfunc(self.g, RS.data.ptr, RS.size,0,1)

            self.func((N_BLOCKS,), (N_THREADS,),
                      (output, self.S0, 
                       self.coupon,self.call_obs,self.call_barrier,
                       self.ki_obs,self.ki_barrier,self.min_val,self.max_val,
                       self.drift,self.stoch,self.df,RS,self.steps,N_PATHS
                      ))
            total += output.mean()
            cp.cuda.stream.get_current_stream().synchronize()
    
        return np.float64(total/N_ROUND*self.notional)

    def _compile(self):
        self.func = cp.RawKernel(r'''
        extern "C" __global__ void autocall (
            float *VT,
            const float S0,
            const float * coupon,
            const bool  * call_obs,
            const float * call_barrier,
            const bool  * ki_obs,
            const float * ki_barrier,
            const float min_val,
            const float max_val,
            const float * drift,
            const float * stoch,
            const float * df,
            const float * bm,
            const long N_STEPS,
            const long N_PATHS)
        {
            unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
            unsigned stride = blockDim.x * gridDim.x;
            unsigned tid = threadIdx.x;

            for (unsigned i = idx; i<N_PATHS; i+=stride)
            {
                float ST = S0;
                unsigned n=0;
                bool called = false;
                bool ki = false;
                for(unsigned n = 0; n < N_STEPS; n++)
                {
                ST *= exp(drift[n] + stoch[n]*bm[i+n*N_PATHS]);
                if(call_obs[n] and ST>=call_barrier[n])
                {
                    called = true;
                    ST = coupon[n]*df[n];
                    break;
                }
                else if(!ki && ki_obs[n] && ST<ki_barrier[n])
                    ki = true;
                }
                if (called)
                    VT[i] = ST;
                else if (ki)
                {
                    if(ST<min_val)
                        ST = min_val;
                    if(ST>max_val)
                        ST = max_val;
                    VT[i] = (ST-S0)*df[N_STEPS-1];
                }
                else
                    VT[i] = coupon[N_STEPS-1]*df[N_STEPS-1];
            }
        }
        ''', 'autocall')

In [0]:
todaydt = dtm(2019,11,20)
expirydt = dtm(2020,11,20)
ir = 0.0260156595595066
vol = 0.114975694793223
N_STEPS = 366
call_barrier = cp.ones(N_STEPS)*1.0
call_idx = [29,60,91,120,151,181,212,242,273,304,334,365]
call_obs = np.isin(np.arange(N_STEPS),call_idx)
coupon = np.zeros(N_STEPS)
coupon[call_idx] = np.asarray(np.arange(1,13)*0.05/12)
ki_barrier = cp.ones(N_STEPS)*0.7
ki_obs = [True if (todaydt+td(days=x+1)).isoweekday()<6 else False for x in range(N_STEPS)]
max_val,min_val = 1.0,0.01

option = EquityOption().with_schedule(todaydt,expirydt)\
            .with_ir(ir).with_vol(vol)
payoff = Snowball(1.0,coupon,call_barrier,call_obs,ki_barrier,ki_obs,min_val,max_val,1e6,cp.float32)\
            .with_proc(*option.calc_term()).with_rng()

In [0]:
N_PATHS = 2048000
dtp = cp.float32

n_threads = 256
n_blocks = (N_PATHS-1) // n_threads + 1
t0=time.time()
print(payoff.run(N_PATHS,4,n_blocks,n_threads))
print(time.time()-t0)

14364.3603515625
0.5493502616882324
